# Small Notebook for:

### 1. Preprocess images to feed Stable Diffusion
### 2. Evaluate how difficult the model grasp the features of your images

## Imports

In [ ]:
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

from src.utils import ImageFolder, InferrenceResult, VAEHandler, denormalize, preprocess_images

## Constants

In [ ]:
BATCH_SIZE = 4

ROOT_DIR = Path("/eval_images_train_difficulty")

MODEL_DIR = ROOT_DIR / "models"

VAE_DIR = MODEL_DIR / "waifu-diffusion-v1-4"
FOCAL_MODEL_DIR = MODEL_DIR / "focal"

IMAGE_SOURCE_DIR = ROOT_DIR / "images"
IMAGE_PREPROCESSED_DIR = ROOT_DIR / "processed"

## Functions

In [ ]:
preprocess_images(
    IMAGE_SOURCE_DIR, 
    IMAGE_PREPROCESSED_DIR, 
    width=512, 
    height=512, 
    focal_model_dir=FOCAL_MODEL_DIR,
)

## Load VAE

In [ ]:
vae_waifu_1_4 = VAEHandler(VAE_DIR)

## Prepare Dataset

In [ ]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(0.5, 1)])
dataset = ImageFolder(IMAGE_PREPROCESSED_DIR, transform).make_iterator(batch_size=BATCH_SIZE, shuffle=False)

## Evaluate

In [ ]:
res = vae_waifu_1_4.calc_loss(dataset)

## Loss

In [ ]:
df = res.df.copy(deep=True)
df.sort_index().plot(x="idx", y="loss", xlabel="image_idx", ylabel="loss", figsize=(7, 5))
plt.show()

## Visualize Results

In [ ]:
res.plot_most_and_least_lossy_images(n=5)